## Mass download the Excels

In [1]:
import requests

In [13]:
base_url = 'https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/'

In [39]:
def download(year, month):
    path = str(year) + '/' + '{:02d}'.format(month) + '/'    
    filename = '4-10-Bil-Tot-Kat-d-J-' + str(year) + '-' + '{:02d}'.format(month) + '.xlsx'
    url = base_url + path + filename
    response = requests.get(url)
    savepath = 'data/migration/'
    with open(savepath + filename, 'wb') as f:
        f.write(response.content)

In [40]:
for year in range(2010, 2017):
    for month in range(3, 15, 3):
        download(year, month)

https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2010/03/4-10-Bil-Tot-Kat-d-J-2010-03.xlsx
https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2010/06/4-10-Bil-Tot-Kat-d-J-2010-06.xlsx
https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2010/09/4-10-Bil-Tot-Kat-d-J-2010-09.xlsx
https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2010/12/4-10-Bil-Tot-Kat-d-J-2010-12.xlsx
https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2011/03/4-10-Bil-Tot-Kat-d-J-2011-03.xlsx
https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2011/06/4-10-Bil-Tot-Kat-d-J-2011-06.xlsx
https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2011/09/4-10-Bil-Tot-Kat-d-J-2011-09.xlsx
https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2011/12/4-10-Bil-Tot-Kat-d-J-2011-12.xlsx
https://www.sem.

In [58]:
for year in range(2017, 2018):
    for month in range(3, 9, 3):
        download(year, month)

https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2017/03/4-10-Bil-Tot-Kat-d-J-2017-03.xlsx
https://www.sem.admin.ch/dam/data/sem/publiservice/statistik/auslaenderstatistik/2017/06/4-10-Bil-Tot-Kat-d-J-2017-06.xlsx


## Mass read the Excel Files

In [66]:
import glob
import pandas as pd
import re

In [133]:
filenames = glob.glob("data/migration/*.xlsx")

In [134]:
filename = []
laufjahr = []
df_all = pd.DataFrame({
    'filename': filename,
    'laufjahr': laufjahr
})

In [135]:
for filename in filenames:
    df = pd.read_excel(filename)
    df_current = pd.DataFrame({
        'filename': [filename],
        'laufjahr': [df.loc[10, 'Unnamed: 1']]
    })
    df_all = df_all.append(df_current)

In [136]:
df_all = df_all.reset_index()

In [137]:
df_all['month'] = df_all['filename'].str.extract(r'(\d{2}).xlsx')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [138]:
df_all['year'] = df_all['filename'].str.extract(r'(\d{4})\-\d{2}.xlsx')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [139]:
del df_all['index']
del df_all['filename']

In [140]:
df_all.to_csv('data/migration_q_laufjahr.csv', index=False)

In [141]:
df_03 = df_all[df_all.month == '03'].reset_index()

In [142]:
df_06 = df_all[df_all.month == '06'].reset_index()
df_09 = df_all[df_all.month == '09'].reset_index()
df_12 = df_all[df_all.month == '12'].reset_index()

In [143]:
df_new = df_03.copy()

In [144]:
del df_new['index']
del df_new['laufjahr']
del df_new['month']

In [145]:
df_new['03'] = df_03['laufjahr']

In [146]:
df_new['06'] = df_06['laufjahr'] - df_03['laufjahr']

In [147]:
df_new['09'] = df_09['laufjahr'] - df_06['laufjahr']

In [148]:
df_new['12'] = df_12['laufjahr'] - df_09['laufjahr']

In [149]:
df_new

,year,03,06,09,12
0,2010,15553.0,13503.0,17052.0,18694.0
1,2011,17707.0,14869.0,22343.0,19219.0
2,2012,18040.0,15980.0,18676.0,20591.0
3,2013,20225.0,20103.0,19312.0,21444.0
4,2014,20054.0,17093.0,19315.0,22432.0
5,2015,22928.0,12206.0,19122.0,17212.0
6,2016,15027.0,13963.0,15344.0,15928.0
7,2017,14635.0,10891.0,NaN,NaN


In [150]:
df_new = df_new.melt(id_vars='year')

In [153]:
df_new = df_new.sort_values(by=['year', 'variable'])

In [154]:
df_new['month'] = df_new['variable'].astype(int) - 1

In [155]:
df_new['month'] = df_new['month'].apply(lambda x: '{:02d}'.format(x))

In [156]:
df_new['date'] = df_new['year'] + "-" + df_new['month'] + "-15"

In [157]:
del df_new['year']
del df_new['variable']
del df_new['month']

In [159]:
df = df_new.copy()

In [160]:
# Only take data starting from 2013 (need to convert first)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.set_index('date')
df = df['2013':].reset_index()

In [161]:
# Convert back to text
df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [163]:
df.drop(df.value.isnull(), inplace=True)

In [164]:
df

,date,value
0,2013-02-15,20225.0
1,2013-05-15,20103.0
2,2013-08-15,19312.0
3,2013-11-15,21444.0
4,2014-02-15,20054.0
5,2014-05-15,17093.0
6,2014-08-15,19315.0
7,2014-11-15,22432.0
8,2015-02-15,22928.0
9,2015-05-15,12206.0


In [165]:
df.to_csv('web/Migration.csv', index=False)

In [168]:
df.value.describe()

count       18.000000
mean     17624.111111
std       3471.785740
min      10891.000000
25%      15106.250000
50%      18167.000000
75%      20090.750000
max      22928.000000
Name: value, dtype: float64

In [169]:
df

,date,value
0,2013-02-15,20225.0
1,2013-05-15,20103.0
2,2013-08-15,19312.0
3,2013-11-15,21444.0
4,2014-02-15,20054.0
5,2014-05-15,17093.0
6,2014-08-15,19315.0
7,2014-11-15,22432.0
8,2015-02-15,22928.0
9,2015-05-15,12206.0
